In [23]:
import math
import time
import torch 
import numpy as np

def arg_min(T, S):
    amin = -1
    m = math.inf  # максимальное значение
    for i, t in enumerate(T):
        if t < m and i not in S:
            m = t
            amin = i

    return amin


def algo_dijk(D):
    N = len(D)  # число вершин в графе
    T = [math.inf]*N   # последняя строка таблицы
    path = list()
    distance_matrix = np.zeros([N, N])
    path_matrix = np.zeros([N, N])
    for node in range(N):
        v = node     # стартовая вершина (нумерация с нуля)
        S = {v}     # просмотренные вершины
        T = [math.inf]*N   # последняя строка таблицы
        T[v] = 0    # нулевой вес для стартовой вершины
        M = [0]*N   # оптимальные связи между вершинами

        while v != -1:          # цикл, пока не просмотрим все вершины
            for j, dw in enumerate(D[v]):   # перебираем все связанные вершины с вершиной v
                if j not in S:           # если вершина еще не просмотрена
                    w = T[v] + dw
                    if w < T[j]:
                        T[j] = w
                        M[j] = v        # связываем вершину j с вершиной v

            v = arg_min(T, S)            # выбираем следующий узел с наименьшим весом
            if v >= 0:                    # выбрана очередная вершина
                S.add(v)                 # добавляем новую вершину в рассмотрение

    #print(T, M, sep="\n")
        
        distance_matrix[node] = T
        path_matrix[node] = M

        
    return(distance_matrix, path_matrix)

def get_all_edges_dijk(path, i, j):
    # формирование оптимального маршрута:
    start = i
    end = j
    P = [j]
    while end != start:
        end = int(path[i][P[-1]])
        P.append(end)
    return P

def prepare_adj(D):
    N = len(D) 
    mask = (torch.ones(N,N, dtype=torch.float) - torch.eye(N,N, dtype=torch.float))*500
    return torch.where(D > 0, D, mask)

In [28]:
def gen_edge_input(max_dist, path, edge_feat):
    # print('start gen_edge_input')
    (nrows, ncols) = path.shape
    assert nrows == ncols
    n = nrows
    max_dist_copy = max_dist

    path_copy = path.astype(long, order='C', casting='safe', copy=True)
    edge_feat_copy = edge_feat.astype(long, order='C', casting='safe', copy=True)
    
    # print('start numpy')
    start_numpy = time.time()
    edge_fea_all = -1 * numpy.ones([n, n, max_dist_copy, edge_feat.shape[-1]], dtype=numpy.int64)
    end_numpy= time.time()
    # print('end numpy')
    # print('numpy end with time', end_numpy - start_numpy)
        
    # print('start sycle')
    start_sycle = time.time()
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if path_copy[i][j] == 510:
                continue
            path = [i] + get_all_edges(path_copy, i, j) + [j]
            # path = [i] + [j]
            num_path = len(path) - 1
            for k in range(num_path):
                # edge_fea_all[i, j, k, :] = edge_feat_copy[path[k], path[k+1], :]
                edge_fea_all[i, j, k, :] = edge_feat_copy[path[0], path[1], :]
                
    end_sycle = time.time()
    # print('end sycle')
    # print('sycle end with time', end_sycle - start_sycle)
    
    # print('end gen_edge_input')
    return edge_fea_all

In [29]:
D = torch.tensor([[0, 3, 1, 3, 0, 0],
     [3, 0, 4, 0, 0, 0],
     [1, 4, 0, 0, 7, 5],
     [3, 0, 0, 0, 0, 2],
     [0, 0, 7, 0, 0, 4],
     [0, 0, 5, 2, 4, 0]], dtype=torch.float)

N = D.size(0)
adj = torch.zeros([N, N], dtype=torch.bool)
adj[edge_index[0, :], edge_index[1, :]] = True
    
D = prepare_adj(D)
distance_matrix = algo_dijk(D)[0]
path_matrix = algo_dijk(D)[1]

get_all_edges_dijk(path_matrix, 5, 0)

max_dist = int(distance_matrix.max())

edge_input = gen_edge_input(max_dist, path_matrix, attn_edge_type.numpy())

NameError: name 'edge_index' is not defined

In [30]:
def gen_edge_input(max_dist, path, edge_feat):
    # print('start gen_edge_input')
    (nrows, ncols) = path.shape
    assert nrows == ncols
    cdef unsigned int n = nrows
    cdef unsigned int max_dist_copy = max_dist

    path_copy = path.astype(long, order='C', casting='safe', copy=True)
    edge_feat_copy = edge_feat.astype(long, order='C', casting='safe', copy=True)
    assert path_copy.flags['C_CONTIGUOUS']
    assert edge_feat_copy.flags['C_CONTIGUOUS']
    
    
    # print('start numpy')
    start_numpy = time.time()
    cdef numpy.ndarray[long, ndim=4, mode='c'] edge_fea_all = -1 * numpy.ones([n, n, max_dist_copy, edge_feat.shape[-1]], dtype=numpy.int64)
    end_numpy= time.time()
    # print('end numpy')
    # print('numpy end with time', end_numpy - start_numpy)
    
    cdef unsigned int i, j, k, num_path, cur
    
    # print('start sycle')
    start_sycle = time.time()
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if path_copy[i][j] == 510:
                continue
            path = [i] + get_all_edges(path_copy, i, j) + [j]
            num_path = len(path) - 1
            for k in range(num_path):
                edge_fea_all[i, j, k, :] = edge_feat_copy[path[k], path[k+1], :]
                
    end_sycle = time.time()
    # print('end sycle')
    # print('sycle end with time', end_sycle - start_sycle)
    
    # print('end gen_edge_input')
    return edge_fea_all

SyntaxError: invalid syntax (2152371971.py, line 5)

In [72]:
b = algo_dijk(D)[1]
b

array([[0., 0., 0., 0., 2., 3.],
       [1., 0., 1., 0., 2., 3.],
       [2., 2., 0., 0., 2., 2.],
       [3., 0., 0., 0., 5., 3.],
       [2., 2., 4., 5., 0., 4.],
       [3., 0., 5., 5., 5., 0.]])

In [2]:
import torch

In [56]:
D = torch.tensor([[0, 1, 1, 0], [1, 0, 0, 0], [1, 0, 0, 1], [0, 0, 1, 0]])
N = len(D) 
D = (torch.ones(N,N) - torch.eye(N,N))*10**5

In [60]:
import numpy as np
D = np.array(D)

In [61]:
def dijkstra(adj):
    n = adj.shape[0]
    distance_matrix = torch.zeros([n, n])
    for i in range(n):
        u = torch.zeros(n, dtype=torch.bool)
        d = np.inf * torch.ones(n)
        d[i] = 0
        q = priorityQ_torch(i)
        while not q.isEmpty():
            v, d_v = q.top()  # point and distance
            v = v.int()
            q.pop()
            if d_v != d[v]:
                continue
            for j, py in enumerate(adj[v]):
                if py == 0 and j != v:
                    continue
                else:
                    to = j
                    weight = py
                    if d[v] + py < d[to]:
                        d[to] = d[v] + py
                        q.push(torch.Tensor([to, d[to]]))
        distance_matrix[i] = d
    return distance_matrix

In [63]:
np.zeros([2,2])

array([0., 0.])

In [15]:
import dijkstra

In [17]:
D = torch.tensor([[0, 3, 1, 3, 0, 0],
     [3, 0, 4, 0, 0, 0],
     [1, 4, 0, 0, 7, 5],
     [3, 0, 0, 0, 0, 2],
     [0, 0, 7, 0, 0, 4],
     [0, 0, 5, 2, 4, 0]], dtype=torch.float)


D = dijkstra.prepare_adj(D)
distance_matrix = dijkstra.algo_dijk(D)[0]
path_matrix = dijkstra.algo_dijk(D)[1]

get_all_edges_dijk(path_matrix, 5, 0)

max_dist = int(distance_matrix.max())

In [18]:
distance_matrix

array([[ 0.,  3.,  1.,  3.,  8.,  5.],
       [ 3.,  0.,  4.,  6., 11.,  8.],
       [ 1.,  4.,  0.,  4.,  7.,  5.],
       [ 3.,  6.,  4.,  0.,  6.,  2.],
       [ 8., 11.,  7.,  6.,  0.,  4.],
       [ 5.,  8.,  5.,  2.,  4.,  0.]])

In [19]:
path_matrix

array([[0., 0., 0., 0., 2., 3.],
       [1., 0., 1., 0., 2., 3.],
       [2., 2., 0., 0., 2., 2.],
       [3., 0., 0., 0., 5., 3.],
       [2., 2., 4., 5., 0., 4.],
       [3., 0., 5., 5., 5., 0.]])

In [20]:
shortest_path_result, path = dijkstra.algo_dijk(D)

In [21]:
shortest_path_result

array([[ 0.,  3.,  1.,  3.,  8.,  5.],
       [ 3.,  0.,  4.,  6., 11.,  8.],
       [ 1.,  4.,  0.,  4.,  7.,  5.],
       [ 3.,  6.,  4.,  0.,  6.,  2.],
       [ 8., 11.,  7.,  6.,  0.,  4.],
       [ 5.,  8.,  5.,  2.,  4.,  0.]])

In [22]:
path

array([[0., 0., 0., 0., 2., 3.],
       [1., 0., 1., 0., 2., 3.],
       [2., 2., 0., 0., 2., 2.],
       [3., 0., 0., 0., 5., 3.],
       [2., 2., 4., 5., 0., 4.],
       [3., 0., 5., 5., 5., 0.]])